In [1]:
import numpy as np
import pandas as pd
import time
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score, mean_squared_error

def preprocess_dataset_train(file_path):
    """
    Train veri setini hazırlar: Eksik değerleri doldurur, kategorik değişkenleri işler,
    log dönüşümü uygular, düşük korelasyonlu sütunları temizler ve IQR yöntemiyle aykırı
    değerleri sınırlar.
    """
    # 📌 1. CSV dosyasını oku
    df = pd.read_csv(file_path)
    print(f"✅ Veri yüklendi: {df.shape}")

    # 🟢 2. Log dönüşümünü uygula (ÖNEMLİ!)
    df["SalePrice"] = np.log1p(df["SalePrice"])
    print("✅ Log dönüşümü uygulandı!")

    # 🟢 3. Eksik değerleri doldur
    df["GarageYrBlt"].fillna(0, inplace=True)
    df["LotFrontage"] = df.groupby("Neighborhood")["LotFrontage"].transform(lambda x: x.fillna(x.median()))
    df["MasVnrArea"].fillna(0, inplace=True)
    print("✅ Eksik değerler dolduruldu!")

    # 🟢 4. Ordinal Encoding Uygula
    ordinal_features = {
        "ExterQual": {"Ex": 5, "Gd": 4, "TA": 3, "Fa": 2, "Po": 1},
        "ExterCond": {"Ex": 5, "Gd": 4, "TA": 3, "Fa": 2, "Po": 1},
        "BsmtQual": {"Ex": 5, "Gd": 4, "TA": 3, "Fa": 2, "Po": 1, "NA": 0},
        "BsmtCond": {"Ex": 5, "Gd": 4, "TA": 3, "Fa": 2, "Po": 1, "NA": 0},
        "KitchenQual": {"Ex": 5, "Gd": 4, "TA": 3, "Fa": 2, "Po": 1},
        "FireplaceQu": {"Ex": 5, "Gd": 4, "TA": 3, "Fa": 2, "Po": 1, "NA": 0},
        "GarageQual": {"Ex": 5, "Gd": 4, "TA": 3, "Fa": 2, "Po": 1, "NA": 0},
        "GarageCond": {"Ex": 5, "Gd": 4, "TA": 3, "Fa": 2, "Po": 1, "NA": 0},
        "PoolQC": {"Ex": 5, "Gd": 4, "TA": 3, "Fa": 2, "NA": 0},
        "Fence": {"GdPrv": 4, "MnPrv": 3, "GdWo": 2, "MnWw": 1, "NA": 0}
    }
    
    for col, mapping in ordinal_features.items():
        df[col] = df[col].map(mapping)

    print("✅ Ordinal Encoding tamamlandı!")

    # 🟢 5. One-Hot Encoding
    categorical_cols = df.select_dtypes(include=['object']).columns
    df = pd.get_dummies(df, columns=categorical_cols, drop_first=True)
    print("✅ One-Hot Encoding tamamlandı!")

    # 🟢 6. SalePrice ile düşük korelasyonlu sütunları kaldır
    correlation_threshold = 0.05
    corr_with_saleprice = df.corr()["SalePrice"].abs()
    low_corr_features = corr_with_saleprice[corr_with_saleprice < correlation_threshold].index
    df.drop(columns=low_corr_features, inplace=True)
    print(f"✅ Düşük korelasyonlu {len(low_corr_features)} sütun kaldırıldı!")

    # 🟢 7. Sayısal değişkenlerdeki eksik değerleri MEDIAN ile doldur
    numerical_cols = df.select_dtypes(include=['int64', 'float64']).columns
    missing_before = df[numerical_cols].isna().sum().sum()
    df[numerical_cols] = df[numerical_cols].apply(lambda x: x.fillna(x.median()))
    print(f"✅ Sayısal eksik değerler median ile dolduruldu! Toplam değiştirilen hücre sayısı: {missing_before}")

    # 🟢 8. Aykırı Değerleri IQR ile Kırp
    def remove_outliers_iqr(df, columns):
        for col in columns:
            Q1 = df[col].quantile(0.25)
            Q3 = df[col].quantile(0.75)
            IQR = Q3 - Q1
            lower_bound = Q1 - 1.5 * IQR
            upper_bound = Q3 + 1.5 * IQR
            df[col] = np.clip(df[col], lower_bound, upper_bound)
        return df

    df = remove_outliers_iqr(df, numerical_cols)
    print("✅ Aykırı değerler IQR yöntemiyle kırpıldı!")

    return df


In [2]:
from sklearn.linear_model import Ridge

def train_ridge_regression(df, alpha=1.0):
    """
    Ridge Regression modeli eğitir ve performans metriklerini ekrana yazdırır.
    Ridge modelinin düzgün çalışması için L2 regulizasyon parametresi olan alpha kullanılabilir.
    """
    # 📌 Hedef değişkeni ve özellikleri belirle
    y = df["SalePrice"]
    X = df.drop(columns=["SalePrice"])

    # 📌 Veriyi Eğitim ve Test Setlerine Bölme
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.001, random_state=42)

    # 📌 Ridge Modelini Başlat ve Eğit
    model = Ridge(alpha=alpha)
    start_time = time.time()
    model.fit(X_train, y_train)

    # 📌 Tahmin Yapma
    y_pred = model.predict(X_test)

    # 📌 Performans Metriklerini Hesapla
    r2 = r2_score(y_test, y_pred)
    rmse_log = np.sqrt(mean_squared_error(np.log1p(y_test), np.log1p(np.maximum(y_pred, 0))))
    elapsed_time = time.time() - start_time

    # 📌 Sonuçları Ekrana Yazdır
    print("\n✅ **Ridge Regression Modeli Eğitildi!**")
    print(f"📊 **R² Score:** {r2:.5f}")
    print(f"📉 **RMSE Log:** {rmse_log:.5f}")
    print(f"⏳ **Training Time:** {elapsed_time:.5f} saniye")
    print(f"🔧 **Alpha:** {alpha}")

    return model


In [3]:
# Train veri setini işle
train_prepared = preprocess_dataset_train("train.csv")

✅ Veri yüklendi: (1460, 81)
✅ Log dönüşümü uygulandı!
✅ Eksik değerler dolduruldu!
✅ Ordinal Encoding tamamlandı!
✅ One-Hot Encoding tamamlandı!
✅ Düşük korelasyonlu 76 sütun kaldırıldı!
✅ Sayısal eksik değerler median ile dolduruldu! Toplam değiştirilen hücre sayısı: 3558
✅ Aykırı değerler IQR yöntemiyle kırpıldı!


C:\Users\lerha\AppData\Local\Temp\ipykernel_12696\3567520513.py:23: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df["GarageYrBlt"].fillna(0, inplace=True)
C:\Users\lerha\AppData\Local\Temp\ipykernel_12696\3567520513.py:25: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example,

In [4]:
# Modeli eğit ve çıktıları al
ridge_regression_model = train_ridge_regression(train_prepared)


✅ **Ridge Regression Modeli Eğitildi!**
📊 **R² Score:** 0.97033
📉 **RMSE Log:** 0.00495
⏳ **Training Time:** 0.02160 saniye
🔧 **Alpha:** 1.0


In [5]:
import numpy as np
import pandas as pd

def preprocess_dataset_test(file_path, train_columns):
    """
    Test veri setini işler ve train setindeki sütunlarla uyumlu hale getirir.
    - Eksik değerleri doldurur.
    - Kategorik değişkenleri işler.
    - One-Hot Encoding uygular.
    - Eksik sütunları sıfır ile doldurur.
    """
    # 📌 1. CSV dosyasını oku
    df = pd.read_csv(file_path)
    print(f"✅ Test verisi yüklendi: {df.shape}")

    # 🟢 2. GarageYrBlt, LotFrontage ve MasVnrArea eksik değerlerini doldur
    df["GarageYrBlt"].fillna(0, inplace=True)
    df["LotFrontage"] = df.groupby("Neighborhood")["LotFrontage"].transform(lambda x: x.fillna(x.median()))
    df["MasVnrArea"].fillna(0, inplace=True)

    print("✅ Eksik değerler dolduruldu!")

    # 🟢 3. Ordinal Encoding Uygula
    ordinal_features = {
        "ExterQual": {"Ex": 5, "Gd": 4, "TA": 3, "Fa": 2, "Po": 1},
        "ExterCond": {"Ex": 5, "Gd": 4, "TA": 3, "Fa": 2, "Po": 1},
        "BsmtQual": {"Ex": 5, "Gd": 4, "TA": 3, "Fa": 2, "Po": 1, "NA": 0},
        "BsmtCond": {"Ex": 5, "Gd": 4, "TA": 3, "Fa": 2, "Po": 1, "NA": 0},
        "KitchenQual": {"Ex": 5, "Gd": 4, "TA": 3, "Fa": 2, "Po": 1},
        "FireplaceQu": {"Ex": 5, "Gd": 4, "TA": 3, "Fa": 2, "Po": 1, "NA": 0},
        "GarageQual": {"Ex": 5, "Gd": 4, "TA": 3, "Fa": 2, "Po": 1, "NA": 0},
        "GarageCond": {"Ex": 5, "Gd": 4, "TA": 3, "Fa": 2, "Po": 1, "NA": 0},
        "PoolQC": {"Ex": 5, "Gd": 4, "TA": 3, "Fa": 2, "NA": 0},
        "Fence": {"GdPrv": 4, "MnPrv": 3, "GdWo": 2, "MnWw": 1, "NA": 0}
    }
    
    for col, mapping in ordinal_features.items():
        if col in df.columns:
            df[col] = df[col].map(mapping)

    print("✅ Ordinal Encoding tamamlandı!")

    # 🟢 4. One-Hot Encoding Uygula
    categorical_cols = df.select_dtypes(include=['object']).columns
    df = pd.get_dummies(df, columns=categorical_cols, drop_first=True)
    print("✅ One-Hot Encoding tamamlandı!")

    # 🟢 5. Train veri setindeki sütunlarla test verisini uyumlu hale getir
    missing_cols = set(train_columns) - set(df.columns)
    for col in missing_cols:
        df[col] = 0  # Eksik sütunları sıfır ile doldur
    
    df = df[train_columns]  # Fazla olan sütunları kaldır

    print(f"✅ Test seti, train setiyle uyumlu hale getirildi! Yeni şekil: {df.shape}")

    # 🟢 6. Eksik Sayısal Değerleri Median ile Doldur
    numerical_cols = df.select_dtypes(include=['int64', 'float64']).columns
    missing_before = df[numerical_cols].isna().sum().sum()
    df[numerical_cols] = df[numerical_cols].apply(lambda x: x.fillna(x.median()))
    print(f"✅ Sayısal eksik değerler median ile dolduruldu! Toplam değiştirilen hücre sayısı: {missing_before}")

    return df


In [6]:
def test_linear_regression(model, test_df, test_csv_path, output_csv="ridge-submission.csv"):
    """
    Eğitilmiş modeli kullanarak test verisi için tahmin yapar ve sonucu Kaggle submission.csv olarak kaydeder.
    """
    # 📌 Test verisini yükle (Id sütununu ayrı al)
    test_raw = pd.read_csv(test_csv_path)
    test_ids = test_raw["Id"]  # Orijinal test verisindeki ID'leri al

    # 📌 Model ile tahmin yap
    predictions_log = model.predict(test_df)

    # 📌 Log dönüşümünü geri al (çünkü train'de log uygulanmıştı)
    predictions = np.expm1(predictions_log)

    # 📌 Tahminleri DataFrame olarak kaydet
    submission = pd.DataFrame({"Id": test_ids, "SalePrice": predictions})

    # 📌 CSV olarak kaydet
    submission.to_csv(output_csv, index=False)
    print(f"✅ Tahminler {output_csv} dosyasına kaydedildi!")

In [7]:
# 📌 1. Test veri setini işle (train veri setiyle uyumlu hale getir)
test_prepared = preprocess_dataset_test("test.csv", train_prepared.drop(columns=["SalePrice"]).columns)

# 📌 2. Modeli kullanarak tahmin yap ve submission.csv dosyasını oluştur
test_linear_regression(ridge_regression_model, test_prepared, "test.csv")


✅ Test verisi yüklendi: (1459, 80)
✅ Eksik değerler dolduruldu!
✅ Ordinal Encoding tamamlandı!
✅ One-Hot Encoding tamamlandı!
✅ Test seti, train setiyle uyumlu hale getirildi! Yeni şekil: (1459, 146)
✅ Sayısal eksik değerler median ile dolduruldu! Toplam değiştirilen hücre sayısı: 3608
✅ Tahminler ridge-submission.csv dosyasına kaydedildi!


C:\Users\lerha\AppData\Local\Temp\ipykernel_12696\3874843467.py:17: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df["GarageYrBlt"].fillna(0, inplace=True)
C:\Users\lerha\AppData\Local\Temp\ipykernel_12696\3874843467.py:19: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example,

In [8]:
asdasd = pd.read_csv("ridge-submission.csv")
asdasd.shape

(1459, 2)